In [40]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np

from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

In [26]:
with open('pogovorki.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')  # убираем первый невидимый символ

In [27]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [28]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('друзья', 6), ('познаются', 3), ('в', 56), ('беде', 3), ('не', 119), ('имей', 4), ('сто', 4), ('рублей', 2), ('а', 56), ('друзей', 8)]


In [29]:
data = tokenizer.texts_to_sequences([texts])
res = to_categorical(data[0], num_classes=maxWordsCount)
print(res.shape)

(2116, 1000)


In [30]:
inp_words = 3
n = res.shape[0] - inp_words

X = np.array([res[i:i + inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [38]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(256, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=50)
model.save("model_w_256_50.h5")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 256)               321792    
                                                                 
 dense_3 (Dense)             (None, 1000)              257000    
                                                                 
Total params: 578,792
Trainable params: 578,792
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
67/67 [==============================] - 1s 5ms/step - loss: 6.7915 - accuracy: 0.0445
Epoch 2/50
67/67 [==============================] - 0s 5ms/step - loss: 6.0618 - accuracy: 0.0611
Epoch 3/50
67/67 [==============================] - 0s 5ms/step - loss: 5.6162 - accuracy: 0.1136
Epoch 4/50
67/67 [==============================] - 0s 5ms/step - loss: 4.9837 - accuracy: 0.2224
Epoch 5/50
67/67 [==============================] - 

In [41]:
model = keras.models.load_model("model_w_256_50.h5")

In [42]:
def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = to_categorical(data[i: i + inp_words], num_classes=maxWordsCount)  # преобразуем в One-Hot-encoding
        inp = x.reshape(1, inp_words, maxWordsCount)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res

In [46]:
res = buildPhrase("март цыплят считает")
print(res)

ValueError: cannot reshape array of size 2000 into shape (1,3,1000)

In [23]:
res = buildPhrase("позитив добавляет годы")
print(res)

1/1 [==============================] - 0s 20ms/step
позитив добавляет годы счастье вашей жизни и двигаться приверженным в вы это в не вы за я и о с все хорошим не


In [25]:
model.save("model_words_text.h5")

In [34]:
model.save("model_w_128_50.h5")